In [37]:
%cd ..
%rm -rf no-time-to-backprop

/content


In [38]:
!ls
!nvidia-smi

global_results.csv  sample_data
Mon Dec 22 19:50:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P0             27W /   70W |     110MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+---------------

In [39]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.80"

!git clone https://github.com/Francis2002/no-time-to-backprop.git
%cd no-time-to-backprop

Cloning into 'no-time-to-backprop'...
remote: Enumerating objects: 314, done.
remote: Counting objects: 100% (176/176), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 314 (delta 108), reused 114 (delta 53), pack-reused 138 (from 1)
Receiving objects: 100% (314/314), 97.92 MiB | 12.67 MiB/s, done.
Resolving deltas: 100% (177/177), done.
/content/no-time-to-backprop


In [40]:
!python -V
!ls

Python 3.12.12
constraints.txt			 requirements-linux-cuda.txt  run_all_toy.sh
hpt.py				 requirements-macos-cpu.txt   run_on_gpu.ipynb
plotting_scripts		 requirements.mac.txt	      run_toy_task.py
requirements-base.txt		 requirements.txt	      test_rec.py
requirements.from_linux_env.txt  run_all_benchmarks.sh	      tgap


In [12]:
%pip install -r requirements-linux-cuda.txt

In [41]:
import jax
import jaxlib
import sys

print(f"jax: {jax.__version__}")
print(f"jaxlib: {jaxlib.__version__}")
print(f"executable: {sys.executable}")
print(f"devices: {jax.devices()}")


jax: 0.8.1
jaxlib: 0.8.1
executable: /usr/bin/python3
devices: [CudaDevice(id=0)]


In [7]:
!python3 run_toy_task.py -m ONLINE -a ZUC --dataset mooc --task link_classification

[*] Results will be stored in /content/no-time-to-backprop/results_for_rotational_vs_none/mooc_ONLINE
[*] d_model set to 64
Running iteration: 0
[*] Output file /content/no-time-to-backprop/tgap/data/npzs/mooc_stream.npz already exists. Using existing file.
object address  : 0x7bd2cbad2f20
object refcount : 3
object type     : 0xa2a4e0
object type name: KeyboardInterrupt
object repr     : KeyboardInterrupt()
lost sys.stderr
^C


In [42]:
# Optuna trials
!python3 run_toy_task.py -m ONLINE -a ZUC --dataset mooc --batching_strategy rearranged --num_epochs 100 --dedupe --hpt optuna --hpt_config ./tgap/benchmarks/hpt_config_example.json


[*] Results will be stored in /content/no-time-to-backprop/results_for_benchmarks/mooc_ONLINE
[*] d_model set to 16
[I 2025-12-22 19:50:36,828] A new study created in memory with name: no-name-830948d1-ddc5-41c0-a53b-310fdbe0d9a9

[*] ----------------------------- Iteration: 0 -----------------------------

[*] Trial: <optuna.trial._trial.Trial object at 0x7e1f667f64e0>
[*] HPT: {'memory': 5, 'seed': 43, 'num_layers': 4, 'state_size': 64, 'd_model_factor': 2.0, 'dropout': 0.22467984684519013, 'pos_weight': 1.0, 'batch_size': 64, 'n_accumulation_steps': 17, 'learning_rate': 0.02148068805313907, 'rec_learning_factor': 1.0, 'beta1': 0.9047263161449228, 'beta2': 0.9740618077985366, 'weight_decay': 0.00013617689706393928}
[*] Npz file /content/no-time-to-backprop/tgap/data/npzs/mooc_stream.npz already exists with matching meta. Using existing file.
W1222 19:50:37.177075   24335 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabr

: 